Линейные модели, которые мы обсуждали на предыдущем занятии, предполагают в качестве зависимой переменной *количественный* признак. Однако, зачастую, наш целевой признак имеет конечное количество значений и является *качественным*. Например, признак *цвет глаз* имеет всего 3 основных значения: голубой, зеленый, карий. В этом случае мы не можем применять линейные модели и говорим, что мы решаем задачу классификации, т.е. классифицируем каждое наблюдение соответствующему значению целевой переменной.

Отметим, что существуют два различных подхода к решению задачи классификации. Первый подход включает в себя методы, которые позволяют сразу относить наблюдение к тому или иному признаку (например, метод ближайших соседей). Второй подход вклюает в себя методы, которые сначала вычисляют вероятность отнесения к какому-либо классу, а потом уже задавая различные пороги отнесения к тому или иному классу, мы принимаем решение к принадлежности класса (например, логистическая регрессия).

Задача классификации встречается на практике обычно даже чаще, чем задача регрессии. Приведем пример некоторых задач:

- Выявление дефолтных заемщиков в банке (банкрот или не банкрот)

- Выявление людей с различными заболеваниями (есть заболевание или нет)

- Мошенническая транзакция или нет 

и т.д.

***В этой лекции прошу обратить внимание на методы kNN и логистическую регрессию. Все остальные методы изучайте чисто для вашего развития, если есть свободное время.***

# 1. Метод ближайших соседей (kNN)

Метод ближайших соседей является одним из самых простых и базовых алгоритмов машинного обучения. "Соседями" в этом методе называются те наблюдения из тестовой выборки, расстояния от которых наименьшее к наблюдениям из тренировочной выборки. Соответственно, задача данного метода заключается в том, чтобы каждому значению целевой переменной из тестовой выборки сопоставить то значение, которое наиболее часто встречается у "соседей" данного наблюдения в тренировочной выборке. Количество соседей здесь - это количество наблюдений из тренировочной выборки, которые наиболее близки к наблюдениям из тестовой выборки.

Расстояние от одного наблюдения к другому может быть измерено несколькими способами (обязательна стандартизация, иначе расстояние будет рассчитываться некорректно):

**Эвклидово расстояние** между точками p = ($p_1$, $p_2$,..., $p_n$) и q = ($q_1$, $q_2$,..., $q_n$):

$$ d(p, q) = d(q, p) = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + ... + (q_n - p_n)^2} $$

**Расстояние Миньковского** порядка $p$ (для p=2 - эвклидово расстояние):

X= $(x_{1},x_{2},\ldots ,x_{n})$ and Y = $(y_{1},y_{2},\ldots ,y_{n}) \in {R}^{n}$:

$$ D\left(X,Y\right)=\left(\sum _{i=1}^{n}|x_{i}-y_{i}|^{p}\right)^{\frac {1}{p}}$$


Давайте попробуем применить данный метод при помощи различных расстояний и количества соседей на таком игрушечном наборе данных:

|Obs. | X1 | X2 | X3 | Y | 
| --  |-- | -- | -- | -- | 
|1 | 0 |3 |0 | Red |
|2 | 2 | 0 | 0| Red |
|3 | 0| 1| 3 | Red | 
|4 | 0| 1| 2| Green | 
|5 |−1 |0 |1 |Green 
|6 | 1| 1| 1| Red

Наша задача предсказать цвет шарика (Y) для точки (0, 0, 0).

In [16]:
import numpy as np
import pandas as pd
y_dict = {"Red": 1, "Green": 0}
X_train = np.array([[0, 3, 0], [2, 0, 0], [0, 1, 3], [0, 1, 2], [-1, 0, 1], [1,1,1]])
y_train = np.array(['Red', 'Red', 'Red', "Green", "Green", "Red"])
X_test = np.array([0, 0, 0])

def minkowsi_dist(v1, v2, p=3):
    """v1, v2: вектора"""
    sum_ = sum(map(lambda v1, v2: np.abs(v1 - v2)**p, v1, v2))
    return np.sign(sum_) * np.abs(sum_) ** (1/p)

# Проверка при помощи готовых реализаций numpy, scipy
dist_mat = np.apply_along_axis(minkowsi_dist, 1, X_train, X_test)
dist_mat_eucl = np.apply_along_axis(minkowsi_dist, 1, X_train, X_test, p=2)
from scipy.spatial import distance
dist_mat_1 = np.apply_along_axis(distance.minkowski, 1, X_train, X_test, p=3)
dist_mat_2 = np.apply_along_axis(distance.minkowski, 1, X_train, X_test)

print(dist_mat)
print(dist_mat_1)
print(dist_mat_eucl)
print(dist_mat_2)

[3.         2.         3.03658897 2.08008382 1.25992105 1.44224957]
[3.         2.         3.03658897 2.08008382 1.25992105 1.44224957]
[3.         2.         3.16227766 2.23606798 1.41421356 1.73205081]
[3.         2.         3.16227766 2.23606798 1.41421356 1.73205081]


In [19]:
dm = dist_mat_1
k = 3
smallest_dist_n = np.argsort(dm)[0:k]
max(y_train[smallest_dist_n])


'Red'

In [37]:
# Переводим цвет шариков в численный тип
y_train_1_0 = pd.Series(y_train).map(y_dict)
y_test_1_0 = pd.Series(y_test).map(y_dict)

# Предскажем цвет шарика при помощи разного количества соседей для эвклидового расстояния.
def predict_knn(y_train, dm, k):
    smallest_dist_n = np.argsort(dm)[0:k]
    return np.round(np.mean(y_train[smallest_dist_n]))
neibors_ = np.arange(1,5)
predicted_knn_mink = dict(zip(neibors_, 
                           list(map(lambda x: predict_knn(y_train_1_0, dist_mat_1, k=x), 
                           neibors_)))
                      )
print(predicted_knn_mink)
predicted_knn_eucl = dict(zip(neibors_, 
                           list(map(lambda x: predict_knn(y_train_1_0, dist_mat_eucl, k=x), 
                           neibors_)))
                      )
print(predicted_knn_eucl)

{1: 0.0, 2: 0.0, 3: 1.0, 4: 0.0}
{1: 0.0, 2: 0.0, 3: 1.0, 4: 0.0}


In [38]:
# Реализация из sklearn
from sklearn.neighbors import KNeighborsClassifier
def sklearn_knn(X_train, y_train, X_test, k):
    knn_skl = KNeighborsClassifier(n_neighbors=k, p=3)
    knn_fit = knn_skl.fit(X_train, y_train)
    knn_predict = knn_fit.predict(X_test.reshape(1,-1))
    return(knn_predict)
predicted_knn_sklearn = dict(zip(neibors_, 
                           list(map(lambda x: sklearn_knn(
                               X_train, y_train, X_test, k=x), 
                           neibors_)))
                      )
predicted_knn_sklearn

{1: array(['Green'], dtype='<U5'),
 2: array(['Green'], dtype='<U5'),
 3: array(['Red'], dtype='<U5'),
 4: array(['Green'], dtype='<U5')}

Теперь попробуем применить ближайших соседей на реальных статистических данных.

Caravan (страховка для трейлеров) датасет включает в себя 85 предиктора, измеряющих различные социально-демографические характеристики для 5822 респондентов. Целевая переменная - *Purchase*, совершил ли индивид покупку страховки Caravan. Отметим, что только 6% людей купило страховку caravan.

In [4]:
import pandas as pd
import os
import glob
os.chdir("/Users/iakubovskii/Machine_Learning/RANEPA/Fintech_2020/Машинное обучение/Данные")
glob.glob("*.csv")
caravan = pd.read_csv("Caravan.csv")
caravan.head()

MOSTYPE  MAANTHUI  MGEMOMV  MGEMLEEF  MOSHOOFD  MGODRK  MGODPR  MGODOV  \
0       33         1        3         2         8       0       5       1   
1       37         1        2         2         8       1       4       1   
2       37         1        2         2         8       0       4       2   
3        9         1        3         3         3       2       3       2   
4       40         1        4         2        10       1       4       1   

   MGODGE  MRELGE  ...  APERSONG  AGEZONG  AWAOREG  ABRAND  AZEILPL  APLEZIER  \
0       3       7  ...         0        0        0       1        0         0   
1       4       6  ...         0        0        0       1        0         0   
2       4       3  ...         0        0        0       1        0         0   
3       4       5  ...         0        0        0       1        0         0   
4       4       7  ...         0        0        0       1        0         0   

   AFIETS  AINBOED  ABYSTAND  Purchase  
0       0        0         0        No  
1       0        0         0        No  
2       0        0         0        No  
3       0        0         0        No  
4       0        0         0        No  

[5 rows x 86 columns]

In [42]:
# Разделим выборку на тренировочную (70%) и тестовую (30%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(caravan.drop("Purchase", axis=1),
                                                   caravan['Purchase'], test_size=0.3,
                                                   random_state = 42 # тип рандома
                                                   )
# Препроцессинг - нормирование признаков
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc_x_train = sc.fit(X_train)
X_train_scal = sc_x_train.transform(X_train)
X_test_scal = sc_x_train.transform(X_test)

def sklearn_caravan_knn(X_train, y_train, X_test, k):
    knn_skl = KNeighborsClassifier(n_neighbors=k, p=3)
    knn_fit = knn_skl.fit(X_train, y_train)
    knn_predict = knn_fit.predict(X_test)
    return(knn_predict)

caravan_predict = sklearn_caravan_knn(X_train_scal, y_train.values, X_test_scal, k=3)

In [6]:
from collections import Counter
print(Counter(caravan_predict))

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, caravan_predict))

Counter({'No': 1718, 'Yes': 29})
0.9232970807097882


In [7]:
caravan_precict_sklearn = dict(zip(neibors_, 
                           list(map(lambda x: Counter(sklearn_caravan_knn(
                               X_train_scal, y_train, X_test_scal, k=x)
                                                     ), 
                           neibors_)
                               )
                                  )
                              )
caravan_precict_sklearn

{1: Counter({'No': 1644, 'Yes': 103}),
 2: Counter({'No': 1737, 'Yes': 10}),
 3: Counter({'No': 1718, 'Yes': 29}),
 4: Counter({'No': 1741, 'Yes': 6})}

In [8]:
def knn_manually(X_train, y_train, X_test, k):
    dist_mat_eucl = np.apply_along_axis(minkowsi_dist, 0, X_train.T, X_test.T, p=2)
    smallest_dist_n = dist_mat_eucl.argpartition(kth = k, axis = 1)[:, 0:k]
    
    return np.apply_along_axis(lambda x: Counter(y_train.iloc[x]).most_common()[0][0], 
                               1, smallest_dist_n)
caravan_precict_manually = dict(zip(neibors_, 
                           list(map(lambda x: Counter(knn_manually(
                               X_train_scal, y_train, X_test_scal, k=x)
                                                     ), 
                           neibors_)
                               )
                                  )
                              )
caravan_precict_manually

{1: Counter({'No': 1642, 'Ye': 105}),
 2: Counter({'No': 1642, 'Ye': 105}),
 3: Counter({'No': 1721, 'Ye': 26}),
 4: Counter({'No': 1725, 'Ye': 22})}

In [44]:
y_train_1_0, y_test_1_0 = y_train.map({"Yes": 1, "No":0}), y_test.map({"Yes": 1, "No":0})
caravan_precict_manually = dict(zip(neibors_, 
                           list(map(lambda x: Counter(knn_manually(
                               X_train_scal, y_train_1_0, X_test_scal, k=x)
                                                     ), 
                           neibors_)
                               )
                                  )
                              )
caravan_precict_manually

{1: Counter({0: 1642, 1: 105}),
 2: Counter({0: 1642, 1: 105}),
 3: Counter({0: 1721, 1: 26}),
 4: Counter({0: 1725, 1: 22})}

In [45]:
y_pred_manually = {k: knn_manually(X_train_scal, y_train_1_0, X_test_scal, k=k) for k in range(1,10)}
y_pred_sklearn = {k: sklearn_caravan_knn(X_train_scal, y_train_1_0, X_test_scal, k=k) for k in range(1,10)}

In [46]:
for k in range(1, 10):
    print(f" Разница в прогнозах для {k} соседей = {sum(y_pred_manually[k] != y_pred_sklearn[k])}")

 Разница в прогнозах для 1 соседей = 30
 Разница в прогнозах для 2 соседей = 97
 Разница в прогнозах для 3 соседей = 19
 Разница в прогнозах для 4 соседей = 20
 Разница в прогнозах для 5 соседей = 4
 Разница в прогнозах для 6 соседей = 8
 Разница в прогнозах для 7 соседей = 4
 Разница в прогнозах для 8 соседей = 3
 Разница в прогнозах для 9 соседей = 0


In [49]:
# Чтобы не тратить время на копирование и вставку почти одних и тех же переменных, можно
# воспользоваться командой globals()
from tqdm.notebook import tqdm
for k in tqdm(range(1, 3)):
    globals()["y_pred_manually_knn_" + str(k)] = knn_manually(
        X_train_scal, y_train_1_0, X_test_scal, k=k)
    globals()["y_pred_sklearn_knn_" + str(k)] = sklearn_caravan_knn(
        X_train_scal, y_train_1_0, X_test_scal, k=k)

In [65]:
KNeighborsClassifier??

***Достоинства и недостатки метода ближайших соседей***

**Достоинства**:

- Простая реализация;
- Хорошо изучена;
- Как правило, метод является хорошим базовым решением не только для классификации или регрессии, но и для рекомендаций;
- Он может быть адаптирован к любой задаче путем выбора правильной метрики. Кстати, [Александр Дьяконов](https://www.kaggle.com/dyakonov), бывший каглер top-1, любит kNN, но с настраиваемой метрикой сходства объектов;
- Хорошая интерпретируемость. Есть исключения: при большом количестве соседей интерпретируемость ухудшается ("Мы ему не давали взаймы, потому что он похож на 350 клиентов, из которых 70 - плохие, а это на 12% больше, чем в среднем по набору данных").

**Недостатки**:

- Метод считается довольно быстрым, но на практике количество соседей, используемых для классификации, обычно велико (100-150), и в этом случае алгоритм будет работать не так быстро.
- Если в наборе данных много переменных, то трудно найти нужные веса и определить, какие из них не важны для классификации/регрессии.
- Зависимость от выбранной метрики расстояния между объектами. Выбор евклидового расстояния по умолчанию часто является необоснованным. Хорошее решение можно найти с помощью поиска по параметрам кросс-валидации, но это становится очень трудоемким для больших наборов данных.
- Теоретических способов выбора количества соседей нет - только поиск по сетке (хотя часто это верно для всех гиперпараметров всех моделей). 
- Как правило, он работает не очень хорошо, когда в модели много признаков из-за "проклятия размерности". Профессор Педро Домингос, известный член сообщества ML, рассказывает об этом [здесь](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf) в своей популярной работе "Немного полезного о машинном обучении"; также "проклятие размерности" описано в книге "Глубокое изучение" в [этой главе](http://www.deeplearningbook.org/contents/ml.html).

# 2. Дискриминативные модели

Традиционно в машинном обучении выделяют два вида моделей, решающих задачу классификации.

Пусть $X$ - признаковое пространство, $y$ - вектор целевой переменной. Тогда вероятностное пространство значений целевой переменной $y$ при заданных значениях признаков $p(x,y) = P(y)p(x|y)$. Читается это так: совместная плотность распределения $x,y$ равняется произведению априорных вероятностей классов к функции правдоподобия классов. 

1. Дискриминативная – моделирует условное распределение $p(x|y)$.

2. Генеративная – моделирует полное распределение $p(x,y)$.

Генеративные модели более общие: если известно $p(x,y)$, то, используя определение
условной вероятности, можно найти $p(x|y)$:

$$ p(x|y) = \frac{p(x,y)}{P(y)} $$

До недавнего времени при решении задач классификации или регрессии обычно использовали дискриминативные модели. Основная причина этого, сформулированная в работе Vapnik, V. N., 1998, заключается в том, что необходимо решать задачу (классификации или регрессии) напрямую, а не решать более общую проблему как промежуточный шаг (например,
моделирование $p(x|y)$ из $p(x, y)$). Но уже в 2001 году было показано, что при использовании дискриминативной модели не во всех задачах достигается меньшая ошибка, чем при использовании генеративной модели. 


## 2.1 Логистическая регрессия

Логистическая регрессия - один из наиболее известных видов дискриминативных моделей и является частным случаем линейного классификатора. Основная идея линейного классификатора состоит в том, что два целевых класса могут быть разделены гиперплоскостью в признаковом пространстве. Если мы можем это сделать без ошибок, тренировочная выборка называется *линейно разделимой*. 
![](https://i1.wp.com/appliedmachinelearning.blog/wp-content/uploads/2017/03/svm_logo1.png?resize=380%2C290&ssl=1)

Мы уже знакомы с линейной регрессией. Давайте попробуем использовать наши знания для решения задачи классификации. Предположим, что наш целевой признак принимает два значения: -1 и +1. Один из наиболее простых способов получить результат классификации, используя линейную регрессию - это применить функцию сигнум:

$$\Large a(\textbf{x}) = \text{sign}(\textbf{w}^\text{T}\textbf x),$$

где
 - $\textbf{x}$ –  признаковое пространство;
 - $\textbf{w}$ – вектор с весами в линейной модели ($w_0$ - константа);
 - $\text{sign}(\bullet)$ – функция, которая возвращает знак аргумента;
 - $a(\textbf{x})$ – отклик классификации при заданных $\textbf{x}$.

Логистическая регрессия - это особый случай линейного классификатора с некоторым преимуществом, которое заключается в том, что мы можем предсказать не только сам класс, но и его вероятность:

$$\Large p_+ = P\left(y_i = 1 \mid \textbf{x}_\text{i}, \textbf{w}\right) $$

Такой подход становится важным, когда мы решаем многие бизнес-задачи: от кредитного скоринга до определения спама. Например, мы выбирает порог $𝑝*$, чтобы предсказать вероятность дефолта по кредиту. Более того, можно умножить эту прогнозируемую вероятность на сумму кредита, чтобы получить ожидаемые убытки от клиента, что также может являться хорошими бизнес-показателями (эксперты по скорингу могут добавить больше, но основная суть в этом). 


Чтобы предсказать вероятность $p_+ \in [0,1]$ мы можем построить линейные предсказания при помощи МНК: $b(\textbf{x}) = \textbf{w}^\text{T} \textbf{x} \in \mathbb{R}$. Затем мы сопоставляем каждому значению вероятность на отрезке [0,1], используя выбранную функцию: $f: \mathbb{R} \rightarrow [0,1]$. В логистической регрессии такая функция называется сигмоида:

$$ \sigma(z) = \frac{1}{1 + \exp^{-z}}$$

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSmTY7ch1O6eaIm_3CuzkhWgeU8SIUoxOPw9w&usqp=CAU)

Предположим, что вероятность наступления события $X$ = $P(X)$. Вспомним, что шанс наступления события - это отношение вероятности к обратной вероятности данного события: $OR(X)_p+ = \frac{P(X)}{1-P(X)}$. По сути, шанс несет в себе абсолютно ту же информацию, что и вероятность. Однако, если вероятность определена на отрезке [0,1], шанс определен на интервале $(0;\infty)$. Например, когда мы говорим, что вероятность выиграть 100 рублей в лотерею равняется 0.01, это означает, что шанс выиграть равен $\frac{0.01}{1-0.01} = \frac{1}{99}$ (в простонародье 99 к 1). 

Если мы возьмем логарифм шанса (логарифм правдоподобия), то  $\log{OR(X)} \in \mathbb{R}$. Теперь посмотрим, как, при помощи логистической регрессии, мы прогнозируем вероятности отнесения к тому или иному классу  $p_+ = P\left(y_i = 1 \mid \textbf{x}_\text{i}, \textbf{w}\right)$:

1. Вычисляем веса $w_{0}+w_{1}x_1 + w_{2}x_2 + ... = \textbf{w}^\text{T}\textbf{x}$. Уравнение $\textbf{w}^\text{T}\textbf{x} = 0$ определяет гиперплоскость, которая разделяет выборку на два класса;

2. Вычисляем логарифм шансов: $ \log(OR_{+}) = \textbf{w}^\text{T}\textbf{x}$

3. Теперь из шансов переходим к вероятностям классов: 

$$\large p_{+} = \frac{OR_{+}}{1 + OR_{+}} = \frac{\exp^{\textbf{w}^\text{T}\textbf{x}}}{1 + \exp^{\textbf{w}^\text{T}\textbf{x}}} = \frac{1}{1 + \exp^{-\textbf{w}^\text{T}\textbf{x}}} = \sigma(\textbf{w}^\text{T}\textbf{x})$$

Видим, что справа у нас стоит сигмоидная функция. Таким образом, логистическая регрессия предсказывает вероятность отнесения к положительному классу при помощи сигмоидной трансформации линейной комбинации весов и признаков:

$$\large p_+(\textbf{x}_\text{i}) = P\left(y_i = 1 \mid \textbf{x}_\text{i}, \textbf{w}\right) = \sigma(\textbf{w}^\text{T}\textbf{x}_\text{i}). $$



Теперь рассмотрим, как вычисляются оценки весов в логистической регрессии. 

Пусть вероятность отнесения к положительному классу:
$$\Large p_+(\textbf{x}_\text{i}) = P\left(y_i = 1 \mid \textbf{x}_\text{i}, \textbf{w}\right) = \sigma(\textbf{w}^T\textbf{x}_\text{i})$$

Пусть вероятность отнесения к отрицательному классу:
$$\Large p_-(\textbf{x}_\text{i})  = P\left(y_i = -1 \mid \textbf{x}_\text{i}, \textbf{w}\right)  = 1 - \sigma(\textbf{w}^T\textbf{x}_\text{i}) = \sigma(-\textbf{w}^T\textbf{x}_\text{i}) $$

Объединяя данные два уравнения, получим:

$$\Large P\left(y = y_i(-1;1) \mid \textbf{x}_\text{i}, \textbf{w}\right) = \sigma(y_i\textbf{w}^T\textbf{x}_\text{i})$$

Выражение $M(\textbf{x}_\text{i}) = y_i\textbf{w}^T\textbf{x}_\text{i}$ называется отсупом классификации для объекта $\textbf{x}_\text{i}$. Если он неотрицательный, то модель корректно предсказывает класс. Если отрицательный, то неверно. Отметим, что отступ определен только для объектов тренировочной выборки, где метка класса известна.

Теперь вычислим правдоподобие нашего датасета, т.е. вероятность наблюдать вектор $\textbf{y}$ в признаковом пространстве $X$ в предположении, что объекты независимо одинаково распределены. Тогда:

$$\Large P\left(\textbf{y} \mid \textbf{X}, \textbf{w}\right) = \prod_{i=1}^{\ell} P\left(y = y_i \mid \textbf{x}_\text{i}, \textbf{w}\right),$$

где $\ell$ количество наблюдений в матрице $\textbf{X}$.
Что такое правдоподобие классно объясняется в [этом видео](https://www.youtube.com/watch?v=pYxNSUDSFH4&t=241s). (осторожно, он иногда поет....)

Далее возьмем логарифм, чтобы преобразовать это в сумму:

$$\Large \log P\left(\textbf{y} \mid \textbf{X}, \textbf{w}\right) = \log \prod_{i=1}^{\ell} P\left(y = y_i \mid \textbf{x}_\text{i}, \textbf{w}\right) = \log \prod_{i=1}^{\ell} \sigma(y_i\textbf{w}^\text{T}\textbf{x}_\text{i})   = $$

$$\Large  = \sum_{i=1}^{\ell} \log \sigma(y_i\textbf{w}^\text{T}\textbf{x}_\text{i}) = \sum_{i=1}^{\ell} \log \frac{1}{1 + \exp^{-y_i\textbf{w}^\text{T}\textbf{x}_\text{i}}} = - \sum_{i=1}^{\ell} \log (1 + \exp^{-y_i\textbf{w}^\text{T}\textbf{x}_\text{i}})$$


Максимизация суммы логарифмов с обратным знаком идентична минимизации:

$$\Large \mathcal{L_{\log}} (\textbf X, \textbf{y}, \textbf{w}) = \sum_{i=1}^{\ell} \log (1 + \exp^{-y_i\textbf{w}^\text{T}\textbf{x}_\text{i}}).$$

Это *логистическая* функция потерь, которая суммируется по всем объектам тренировочной выборки. 

Для сравнения функция потерь в случае линейной регрессии выглядит вот так:

$$\Large \mathcal{L} (\textbf X, \textbf{y}, \textbf{w}) = (\textbf{y} - \textbf{w}\textbf{X})^T (\textbf{y} - \textbf{w}\textbf{X}) = 
\sum_{i=1}^{\ell} (\textbf{y}_i - \textbf{w}\textbf{X}_i) (\textbf{y}_i - \textbf{w}\textbf{X}_i)$$


**Регуляризация для логистической регрессии**

Вместо минимизации функции $\mathcal{L_{\log}} (\textbf X, \textbf{y}, \textbf{w})$ мы минимизируем следующую функцию потерь:

$$\Large \mathcal{J}(\textbf X, \textbf{y}, \textbf{w}) = \mathcal{L_{\log}} (\textbf X, \textbf{y}, \textbf{w}) + \lambda |\textbf{w}|^2$$

В случае логистической регрессии, коэффициент обратной регуляризации обычно используют $C = \frac{1}{\lambda}$. Тогда решения нашей задачи имеет вид:

$$\Large \widehat{\textbf w}  = \arg \min_{\textbf{w}} \mathcal{J}(\textbf X, \textbf{y}, \textbf{w}) =  \arg \min_{\textbf{w}}\ (C\sum_{i=1}^{\ell} \log (1 + \exp^{-y_i\textbf{w}^\text{T}\textbf{x}_\text{i}})+ |\textbf{w}|^2)$$ 

**Темы, изложенные ниже, вы можете изучить самостоятельно.**

## 2.2  Метод опорных векторов

## 2.3 Перцептрон

# 3. Генеративные модели

## 3.1 Наивный Байесовский классификатор

## 3.2 Линейный дискриминантный анализ

## 3.3 Квадратичный дискриминантный анализ